In [ ]:
from rk1pert import *
import matplotlib.pyplot as plt
%matplotlib inline

n = 5
dmin = 0
dmax = 20
tmin = 0
tmax = 20
tstep = .01
plot_asymptotes = True

D = rand_diag(n, dmin, dmax)
u = rand_unit_vector(n)
U = np.outer(u, u)
print("u = " + str(u))
print("D = " + str(np.diag(D)))
A = pencil(D, U)

x, y = get_data_points(A, tmin, tmax, tstep)
for i in range(n):
    plt.plot(x, y[i])
plt.xlabel(r"$t$")
plt.ylabel(r"Eigenvalues of $A(t)$")

# Asymptotes
if plot_asymptotes:
    U, s, V = np.linalg.svd(U) # U will now contain an extension of u to an orthonormal basis for R^n
    U = U[:, 1:]
    S = U.T.dot(D).dot(U)
    mu = np.linalg.eigvalsh(S)
    mu.sort()
    mu = mu[::-1]
    print(u"\u03bc = " + str(mu))
    for y in mu:
        plt.plot(x, y * np.ones(len(x)), ':')
    
    plt.plot(x, D.dot(u).dot(u) + x, ':')

plt.text(0, -10, "u = " + str(u) + "\nD = " + str(np.diag(D)) + "\nmu = " + str(mu))
plt.savefig("output.pdf", bbox_inches = "tight") # This makes the text visible in the PDF
plt.show()

np.savez("output.npz", u = u, d = np.diag(D))